## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Populate the BigQuery Event table


### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- This notebook will download the current day events from Google Events for each city.

- Notebook Logic:
    1. For each city
        - Get a list of the events 
        - Append this to an overall list
    2. Bulk insert the data into BigQuery
    3. Show the results

## Initialize Python

In [ ]:
# You will need to create an account and verify your email.
# https://serpapi.com/
# You get 100 free calls (per month)

serpapi_key = "<<Get your own key>>"

In [ ]:
# We will generate events for each city for the number of days.  Since we have 4 cites and 100 free calls, you need to do at most 25 days.
# date:today - Today's Events
# date:tomorrow - Tomorrow's Events
htichips = "date:today"

# No need to set these
city_names=["New York City", "London", "Tokyo", "San Francisco"]
city_ids=[1,2,3,4]
city_timezones=["US/Eastern","Europe/London","Asia/Tokyo","US/Pacific"]
number_of_coffee_trucks = "5"

In [ ]:
import sys
!{sys.executable} -m pip install google-search-results

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

## Create the GenAI Insights table

In [ ]:
%%bigquery
CREATE TABLE IF NOT EXISTS `${project_id}.${bigquery_data_beans_curated_dataset}.event`
--CREATE OR REPLACE TABLE `${project_id}.${bigquery_data_beans_curated_dataset}.event` -- only use this for starting over
(
    event_id          INT     NOT NULL OPTIONS(description="The primary key of the event."),
    city_id           INT     NOT NULL OPTIONS(description="The foreign key of the city."),
    event_title       STRING  NOT NULL OPTIONS(description="The title of the event."),
    event_date        DATE    NOT NULL OPTIONS(description="The date of the event."),
    event_time_string STRING  NOT NULL OPTIONS(description="The time (string value)  of the event."),
    event_venue       STRING  NOT NULL OPTIONS(description="The venue of the event."),
    event_venue_link  STRING  NOT NULL OPTIONS(description="The generated insight in text"),
    event_address     STRING  NOT NULL OPTIONS(description="The full address of the event."),
    event_description STRING  NOT NULL OPTIONS(description="The description of the event."),
    event_reviews     INT64   NOT NULL OPTIONS(description="The number of reviews of the event."),
    event_thumbnail   STRING  NOT NULL OPTIONS(description="The thumbnail image for the event.")
)
CLUSTER BY event_id;

## Supporting Functions

In [ ]:
def RunQuery(sql):
  import time

  if (sql.startswith("SELECT")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

In [ ]:
def GetNextPrimaryKey(fully_qualified_table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{fully_qualified_table_name}`
  """
  # print(sql)
  df_result = client.query(sql).to_dataframe()
  # display(df_result)
  return df_result['result'].iloc[0] + 1

## Get the list of Events

You can use a 3rd party service to download the events
- https://serpapi.com/google-events-api
- https://serpapi.com/blog/scrape-google-events-results-with-python/

In [ ]:
from serpapi import GoogleSearch
from datetime import date

event_id = GetNextPrimaryKey("${project_id}.${bigquery_data_beans_curated_dataset}.event","event_id")
print(f"event_id: {event_id}")

event_date = date.today()
event_records = []

# Loop for each city
for city_index in range(0, 4):
  print(f"City: {city_ids[city_index]}")

  google_events_params = {
    "engine": "google_events",
    "q": f"Events in {city_names[city_index]}",
    "hl": "en",
    "gl": "us",
    "api_key": f"{serpapi_key}",
    'start': 0,
    "htichips" : f"{htichips}"
  }

  while True:
      search = GoogleSearch(google_events_params)
      event_search_results = search.get_dict()
      if 'error' in event_search_results:
          break

      for item in event_search_results["events_results"]:
          try:
              event_venue = ""
              address = ""
              for address_item in item["address"]:
                if event_venue == "":
                  split_text = address_item.split(", ")
                  event_venue = split_text[0]
                  address = split_text[1]
                else:
                  address = address + " " + address_item

              #print(f"item: {item}")
              #print(f"event_venue: {event_venue}")
              #print(f"address: {address}")
              #print("------------------------------------------------------")

              event = {
                  "event_id" : event_id,
                  "city_id" : city_ids[city_index],
                  "event_title" : item['title'],
                  "event_date" : event_date,
                  "event_time_string" : item['date']['when'],
                  "event_venue" : event_venue,
                  "event_venue_link" : item['venue']['link'],
                  "event_address" : address,
                  "event_description" : item['description'],
                  "event_reviews" : item['venue']['reviews'],
                  "event_thumbnail" : item['thumbnail']
              }
              event_id = event_id + 1

              event_records.append(event)

          except Exception as error:
              print(f"error: {error}")
              print(f"item: {item}")
              print(f"------------------------------")

      google_events_params['start'] += 10

      if google_events_params['start'] > 30:
          break # only do 30 for now so our prompt is not too large

In [ ]:
event_records

## Insert the events (in bulk)

In [ ]:
%%bigquery

-- remove the current days events (in case you re-run this notebook)
DELETE
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.event` AS event
 WHERE event_date = CURRENT_DATE();

In [ ]:
import pandas as pd

# Load the events table (in bulk)
table_id = "${project_id}.${bigquery_data_beans_curated_dataset}.event"

dataframe = pd.DataFrame(
    pd.DataFrame(event_records), # Your source data
    columns=[
        "event_id",
        "city_id",
        "event_title",
        "event_date",
        "event_time_string",
        "event_venue",
        "event_venue_link",
        "event_address",
        "event_description",
        "event_reviews",
        "event_thumbnail"
    ],
)

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("event_id", bigquery.enums.SqlTypeNames.INT64, mode="REQUIRED"),
        bigquery.SchemaField("city_id", bigquery.enums.SqlTypeNames.INT64, mode="REQUIRED"),
        bigquery.SchemaField("event_title", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED"),
        bigquery.SchemaField("event_date", bigquery.enums.SqlTypeNames.DATE, mode="REQUIRED"),
        bigquery.SchemaField("event_time_string", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED"),
        bigquery.SchemaField("event_venue", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED"),
        bigquery.SchemaField("event_venue_link", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED"),
        bigquery.SchemaField("event_address", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED"),
        bigquery.SchemaField("event_description", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED"),
        bigquery.SchemaField("event_reviews", bigquery.enums.SqlTypeNames.INT64, mode="REQUIRED"),
        bigquery.SchemaField("event_thumbnail", bigquery.enums.SqlTypeNames.STRING, mode="REQUIRED")
    ],
    write_disposition="WRITE_APPEND",
)

job = client.load_table_from_dataframe(dataframe, table_id, job_config=job_config)
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))

## See the results

In [ ]:
%%bigquery

# Display the results
SELECT *
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.event`
WHERE event_date = CURRENT_DATE()
ORDER BY event_id